<a href="https://colab.research.google.com/github/chenchongsong/udacity-cs344-colab/blob/main/notebook/udacity_cs344_hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Homework 6 for Udacity CS344 Course, Intro to Parallel Programming
# clone the code repo,
!git clone https://github.com/chenchongsong/udacity-cs344-colab
!pip install git+git://github.com/depctg/nvcc4jupyter.git

# load cuda plugin
%config NVCCPluginV2.static_dir = True
%config NVCCPluginV2.relative_dir = "udacity-cs344-colab/src/HW6"
%load_ext nvcc_plugin

# change to work directory, generate makefiles
!mkdir udacity-cs344-colab/build
%cd udacity-cs344-colab/build
!cmake ../src

In [ ]:
%%cuda --name student_func.cu

//Udacity HW 6
//Poisson Blending

/* Background
   ==========

   The goal for this assignment is to take one image (the source) and
   paste it into another image (the destination) attempting to match the
   two images so that the pasting is non-obvious. This is
   known as a "seamless clone".

   The basic ideas are as follows:

   1) Figure out the interior and border of the source image
   2) Use the values of the border pixels in the destination image
      as boundary conditions for solving a Poisson equation that tells
      us how to blend the images.

      No pixels from the destination except pixels on the border
      are used to compute the match.

   Solving the Poisson Equation
   ============================

   There are multiple ways to solve this equation - we choose an iterative
   method - specifically the Jacobi method. Iterative methods start with
   a guess of the solution and then iterate to try and improve the guess
   until it stops changing.  If the problem was well-suited for the method
   then it will stop and where it stops will be the solution.

   The Jacobi method is the simplest iterative method and converges slowly -
   that is we need a lot of iterations to get to the answer, but it is the
   easiest method to write.

   Jacobi Iterations
   =================

   Our initial guess is going to be the source image itself.  This is a pretty
   good guess for what the blended image will look like and it means that
   we won't have to do as many iterations compared to if we had started far
   from the final solution.

   ImageGuess_prev (Floating point)
   ImageGuess_next (Floating point)

   DestinationImg
   SourceImg

   Follow these steps to implement one iteration:

   1) For every pixel p in the interior, compute two sums over the four neighboring pixels:
      Sum1: If the neighbor is in the interior then += ImageGuess_prev[neighbor] (A)
             else if the neighbor in on the border then += DestinationImg[neighbor] (B)

      Sum2: += SourceImg[p] - SourceImg[neighbor]   (for all four neighbors) (C)

   2) Calculate the new pixel value:
      float newVal= (Sum1 + Sum2) / 4.f  <------ Notice that the result is FLOATING POINT
      ImageGuess_next[p] = min(255, max(0, newVal)); //clamp to [0, 255]


    In this assignment we will do 800 iterations.
   */



#include "utils.h"

const unsigned int NUM_COLS = 500;
const unsigned int NUM_ROWS = 333;
const unsigned int BLOCK_ROWS = 5;

__device__
bool isNotWhite(const uchar4 in) {
  return (in.x + in.y + in.z < 3 * 255);
}

__global__ 
void blendImage(const unsigned char* const mask,
                float* const red,
                float* const blue,
                float* const green,
                uchar4* const dest) {

  unsigned int row = blockIdx.x;
  unsigned int col = threadIdx.x;
  unsigned int idx = col + row * NUM_COLS; // global index 

  if ( row >= NUM_ROWS || col >= NUM_COLS ) return;

  if ( mask[idx] < 2 ) return;  // for out-of-mask region & border region, same as original swimming pool

  dest[idx].x = (unsigned char)red[idx];
  dest[idx].y = (unsigned char)blue[idx];
  dest[idx].z = (unsigned char)green[idx];
}

__global__
void preProcessSource(const uchar4* const in,
                      const uchar4* const dest,
                      const unsigned char* const mask,
                      float* const red,
                      float* const blue,
                      float* const green ) {

  unsigned int row = blockIdx.x;
  unsigned int col = threadIdx.x;
  unsigned int idx = col + row * NUM_COLS; // global index 

  if ( row >= NUM_ROWS || col >= NUM_COLS ) return;

  switch ( mask[idx] ) {
    case 0:  // out of mask
      red[idx] = 0; blue[idx] = 0; green[idx] = 0;
      break;
    case 1:  // border region, init with swimming pool values
      red[idx] = dest[idx].x;
      blue[idx] = dest[idx].y;
      green[idx] = dest[idx].z;
      break;
    case 2:  // interior region, init with polar bear values
      red[idx] = in[idx].x;
      blue[idx] = in[idx].y;
      green[idx] = in[idx].z;
  }
}

__global__
void separateIntoChannels(const uchar4 * const in,
                          float * const red,
                          float * const blue,
                          float * const green ) {

  unsigned int row = blockIdx.x;
  unsigned int col = threadIdx.x;
  unsigned int idx = col + row * NUM_COLS; // global index 

  if ( row >= NUM_ROWS || col >= NUM_COLS ) return;

  red[idx] = in[idx].x;
  blue[idx] = in[idx].y;
  green[idx] = in[idx].z;
}

__global__
void makeMask( const uchar4* const in,
               unsigned char * const out) {
  // the source image is addresses in column-major order

  __shared__ unsigned char img[3 * NUM_COLS];
  __shared__ short neighbors[4];

  unsigned int row = blockIdx.x;
  unsigned int col = threadIdx.x;
  unsigned int idx = col + row * NUM_COLS; // global index 
  unsigned int sIdx = col + NUM_COLS;  // index in shared array

  if ( row >= NUM_ROWS || col >= NUM_COLS ) return;

  neighbors[0] = -1;
  neighbors[1] = 1;
  neighbors[2] = -NUM_COLS;
  neighbors[3] = NUM_COLS;
    
  img[sIdx - NUM_COLS] = row > 0 ? isNotWhite(in[idx - NUM_COLS]) : 0;
  img[sIdx] = isNotWhite(in[idx]) ? 1 : 0;
  img[sIdx + NUM_COLS] = row + 1 < NUM_ROWS ? isNotWhite(in[idx + NUM_COLS]) : 0;
  __syncthreads();

  if ( img[sIdx] == 0 ) return;

  unsigned char acc = 0;
  for ( short i = 0; i < 4 ; i++ ) acc += img[sIdx + neighbors[i]];

  out[idx] = acc < 4 ? 1 : 2;  // 0: out of mask, 1: border, 2: interior
}

__global__
void computeMetric(const float* const in,
                   const unsigned char* const mask,
                   float* const out) {

  __shared__ unsigned char img[3 * NUM_COLS];
  __shared__ short neighbors[4]; 

  unsigned int row = blockIdx.x;
  unsigned int col = threadIdx.x;
  unsigned int idx = col + row * NUM_COLS; // global index 
  unsigned int sIdx = col + NUM_COLS;  // index in shared array
 
  if ( row >= NUM_ROWS || col >= NUM_COLS ) return;

  neighbors[0] = -1;
  neighbors[1] = 1;
  neighbors[2] = -NUM_COLS;
  neighbors[3] = NUM_COLS;
 
  img[sIdx - NUM_COLS] = row > 0 ? in[idx - NUM_COLS] : 0;
  img[sIdx] = in[idx];
  img[sIdx + NUM_COLS] = row + 1 < NUM_ROWS ? in[idx + NUM_COLS] : 0;
  __syncthreads();

  if ( mask[idx] < 2 ) return;

  float acc = 4.f * img[sIdx];  // Sum2: += SourceImg[p] - SourceImg[neighbor] (for all four neighbors) (C)
  for ( short i = 0; i < 4; i++ ) acc -= img[sIdx + neighbors[i]];

  out[idx] = acc;
}

__global__
void computeIterate(const unsigned char* const mask,
                    float* const in,
                    float* const out,
                    float* const metric) {

  __shared__ float img[(1 + BLOCK_ROWS + 1) * NUM_COLS];

  // current block is responsible for rows "[blockIdx.x, blockIdx.x + BLOCK_ROWS)"
  // each thread is responsible for one column
  unsigned int col = threadIdx.x;
  unsigned int row_start = blockIdx.x * BLOCK_ROWS;
  unsigned int row_end = row_start + BLOCK_ROWS > NUM_ROWS ? NUM_ROWS : row_start + BLOCK_ROWS;
  // if ( col >= NUM_COLS ) return;

  for (unsigned int row = row_start; row < row_end; row++) {  // global row id
    unsigned int idx = col + row * NUM_COLS;  // global index
    unsigned int sIdx = col + (row - row_start + 1) * NUM_COLS;  // index in shared array
    if (row == row_start) {
      img[sIdx - NUM_COLS] = row == 0 ? 0 : in[idx - NUM_COLS];
    }
    img[sIdx] = in[idx];
    if (row == row_end - 1) {
      img[sIdx + NUM_COLS] = row == NUM_ROWS - 1 ? 0 : in[idx + NUM_COLS];
    }
  }
  __syncthreads();  // sync the whole block

  for (unsigned int row = row_start; row < row_end; row++) {  // global row id
    unsigned int idx = col + row * NUM_COLS;  // global index

    if ( mask[idx] == 2 ) {
      unsigned int sIdx = col + (row - row_start + 1) * NUM_COLS;  // index in shared array
      float acc = img[sIdx - 1] + img[sIdx + 1] + img[sIdx - NUM_COLS] + img[sIdx + NUM_COLS];
      acc = (acc + metric[idx]) / 4.f;
      out[idx] = fminf(255.f, fmaxf(0.f, acc));
    }
  }
}

void your_blend(const uchar4* const h_sourceImg,  //IN
                const size_t numRowsSource, const size_t numColsSource,
                const uchar4* const h_destImg, //IN
                uchar4* const h_blendedImg //OUT
) {
  /* To Recap here are the steps you need to implement

     1) Compute a mask of the pixels from the source image to be copied
        The pixels that shouldn't be copied are completely white, they
        have R=255, G=255, B=255.  Any other pixels SHOULD be copied.

     2) Compute the interior and border regions of the mask.  An interior
        pixel has all 4 neighbors also inside the mask.  A border pixel is
        in the mask itself, but has at least one neighbor that isn't.

     3) Separate out the incoming image into three separate channels

     4) Create two float(!) buffers for each color channel that will
        act as our guesses.  Initialize them to the respective color
        channel of the source image since that will act as our intial guess.

     5) For each color channel perform the Jacobi iteration described
        above 800 times.

     6) Create the output image by replacing all the interior pixels
        in the destination image with the result of the Jacobi iterations.
        Just cast the floating point values to unsigned chars since we have
        already made sure to clamp them to the correct range.

      Since this is final assignment we provide little boilerplate code to
      help you.  Notice that all the input/output pointers are HOST pointers.

      You will have to allocate all of your own GPU memory and perform your own
      memcopies to get data in and out of the GPU memory.

      Remember to wrap all of your calls with checkCudaErrors() to catch any
      thing that might go wrong.  After each kernel call do:

      cudaDeviceSynchronize(); checkCudaErrors(cudaGetLastError());

      to catch any errors that happened while executing the kernel.
  */

  const unsigned int srcSize = numRowsSource * numColsSource;
  uchar4 * d_sourceImg, *d_destImg;
  unsigned char *d_mask;
  float *d_blended_red_in, *d_blended_red_out, *d_blended_green_in, 
        *d_blended_green_out, *d_blended_blue_in, *d_blended_blue_out, 
        *d_dest_red, *d_dest_green, *d_dest_blue, *d_metric_red,
        *d_metric_blue, *d_metric_green;

  checkCudaErrors(cudaMalloc(&d_sourceImg, sizeof(uchar4) * srcSize));
  checkCudaErrors(cudaMemset(d_sourceImg, 0, sizeof(uchar4) * srcSize));

  checkCudaErrors(cudaMalloc(&d_destImg, sizeof(uchar4) * srcSize));
  checkCudaErrors(cudaMemset(d_destImg, 0, sizeof(uchar4) * srcSize));

  checkCudaErrors(cudaMalloc(&d_mask, sizeof(unsigned char) * srcSize));
  checkCudaErrors(cudaMemset(d_mask, 0, sizeof(unsigned char) * srcSize));

  checkCudaErrors(cudaMalloc(&d_blended_red_in, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_blended_red_in, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_blended_green_in, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_blended_green_in, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_blended_blue_in, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_blended_blue_in, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_blended_red_out, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_blended_red_out, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_blended_green_out, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_blended_green_out, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_blended_blue_out, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_blended_blue_out, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_dest_red, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_dest_red, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_dest_green, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_dest_green, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_dest_blue, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_dest_blue, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_metric_red, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_metric_red, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_metric_blue, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_metric_blue, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMalloc(&d_metric_green, sizeof(float) * srcSize));
  checkCudaErrors(cudaMemset(d_metric_green, 0, sizeof(float) * srcSize));

  checkCudaErrors(cudaMemcpy(d_sourceImg, h_sourceImg, sizeof(uchar4) * srcSize, cudaMemcpyHostToDevice));
  checkCudaErrors(cudaMemcpy(d_destImg, h_destImg, sizeof(uchar4) * srcSize, cudaMemcpyHostToDevice));

  // makeMask & separateIntoChannels can be done simultanouesly.
  makeMask<<<NUM_ROWS, NUM_COLS>>>(d_sourceImg, d_mask);
  separateIntoChannels<<<NUM_ROWS, NUM_COLS>>>(d_sourceImg, d_blended_red_in, d_blended_blue_in, d_blended_green_in);

  computeMetric<<<NUM_ROWS, NUM_COLS>>>(d_blended_red_in, d_mask, d_metric_red);
  computeMetric<<<NUM_ROWS, NUM_COLS>>>(d_blended_blue_in, d_mask, d_metric_blue);
  computeMetric<<<NUM_ROWS, NUM_COLS>>>(d_blended_green_in, d_mask, d_metric_green);

  preProcessSource<<<NUM_ROWS, NUM_COLS>>>(d_sourceImg, d_destImg, d_mask, d_blended_red_in, d_blended_blue_in, d_blended_green_in);
  preProcessSource<<<NUM_ROWS, NUM_COLS>>>(d_sourceImg, d_destImg, d_mask, d_blended_red_out, d_blended_blue_out, d_blended_green_out);

  const unsigned int gridSize = (NUM_ROWS + BLOCK_ROWS - 1) / BLOCK_ROWS;  // each block is responsible for BLOCK_ROWS x NUM_COLS
  for(short k = 0; k < 800; k++) {
    computeIterate<<<gridSize, NUM_COLS>>>(d_mask, d_blended_red_in, d_blended_red_out, d_metric_red);
    computeIterate<<<gridSize, NUM_COLS>>>(d_mask, d_blended_blue_in, d_blended_blue_out, d_metric_blue);
    computeIterate<<<gridSize, NUM_COLS>>>(d_mask, d_blended_green_in, d_blended_green_out, d_metric_green);
    // double buffering
    std::swap(d_blended_red_in, d_blended_red_out);
    std::swap(d_blended_blue_in, d_blended_blue_out);
    std::swap(d_blended_green_in, d_blended_green_out);
  }

  blendImage<<<NUM_ROWS, NUM_COLS>>>(d_mask, d_blended_red_in, d_blended_blue_in, d_blended_green_in, d_destImg);
  checkCudaErrors(cudaMemcpy(h_blendedImg, d_destImg, sizeof(uchar4) * srcSize, cudaMemcpyDeviceToHost));

  checkCudaErrors(cudaFree(d_mask));
  checkCudaErrors(cudaFree(d_blended_red_in));
  checkCudaErrors(cudaFree(d_blended_blue_in));
  checkCudaErrors(cudaFree(d_blended_green_in));
  checkCudaErrors(cudaFree(d_blended_red_out));
  checkCudaErrors(cudaFree(d_blended_blue_out));
  checkCudaErrors(cudaFree(d_blended_green_out));
  checkCudaErrors(cudaFree(d_metric_red));
  checkCudaErrors(cudaFree(d_metric_blue));
  checkCudaErrors(cudaFree(d_metric_green));
  checkCudaErrors(cudaFree(d_dest_red));
  checkCudaErrors(cudaFree(d_dest_blue));
  checkCudaErrors(cudaFree(d_dest_green));
  checkCudaErrors(cudaFree(d_sourceImg));
  checkCudaErrors(cudaFree(d_destImg));
}

In [ ]:
# make the cuda project
!nvidia-smi
!make HW6
print("\n====== RESULT OF HW6 =======\n")
!bin/HW6 ../src/HW6/source.png ../src/HW6/blended.gold

In [ ]:
# plot output images
import matplotlib.pyplot as plt
_,ax = plt.subplots(2,3, dpi=150)

ax[0][0].imshow(plt.imread("../src/HW6/source.png"))
ax[0][0].set_title("src: bear")
ax[0][0].grid(False)

ax[0][1].imshow(plt.imread("../src/HW6/destination.png"))
ax[0][1].set_title("dest: pool")
ax[0][1].grid(False)

ax[0][2].imshow(plt.imread("HW6_output.png"))
ax[0][2].set_title("output")
ax[0][2].grid(False)

ax[1][0].imshow(plt.imread("HW6_reference.png"))
ax[1][0].set_title("reference")
ax[1][0].grid(False)

ax[1][1].imshow(plt.imread("HW6_differenceImage.png"))
ax[1][1].set_title("difference")
ax[1][1].grid(False)

# leave ax[1][2] as blank

plt.show()